## GPU mátrix szorzás

GPU-s dolgokat elég macerás telepíteni, a Cuda eleve csak Nvidia kártyákra van, ezért [Google Colabban](https://colab.research.google.com/) futtattam ezeket a kódokat.

Ekkor szükséges az alábbi beállítás: *Futtatókörnyezet/Futtatókörnyezet módosítása/Hardveres gyorsítás: GPU*

In [4]:
"""Könyvtárak importálása"""

import math
import numpy as np
from numba import cuda, float32
import tensorflow as tf

In [5]:
"""Mátrixok létrehozása"""

n = 256
a = np.random.random((n, n)).astype(np.float32)
b = np.random.random((n, n)).astype(np.float32)
c = np.zeros((n, n), dtype=np.float32)

In [6]:
"""CPU Referencia: numpy.matmul"""

%timeit np.matmul(a, b)

1.39 ms ± 491 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
"""GPU szorzás CUDA-val (nem saját kód)"""
# forráskód: https://numba.readthedocs.io/en/stable/cuda/examples.html

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
# TPB should not be larger than 32 in this example
TPB = 32

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

In [12]:
# tömbök átmásolása GPU-ra
a_d = cuda.to_device(a)
b_d = cuda.to_device(b)
c_d = cuda.to_device(c)

# A GPU thredjeinek Blockokba és Gridekbe osztása
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(c.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(c.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

# mátrixszorzás
%timeit fast_matmul[blockspergrid, threadsperblock](a_d, b_d, c_d)

# visszamásolás memóriába
c = c_d.copy_to_host()  # Ez sokáig tarthat

87.6 µs ± 43.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
"""GPU szorzás Tensorflow segítségével"""

# tömbök átmásolása GPU-ra
a_tensor = tf.constant(a)
b_tensor = tf.constant(b)

# mátrixszorzás beépített paranccsal
c_tensor = tf.matmul(a_tensor, b_tensor)
%timeit tf.matmul(a_tensor, b_tensor)

# visszamásolás memóriába
c = c_tensor.numpy()  # Ez sokáig tarthat

121 µs ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Eredmények
A GPU-s esetben igen nagy a szórás. Néha nagyságrendekkel tovább tart. Nagy méretnél a tömbök átmásolása jóval hosszabb idő, mint maga a szorzás.


| n     	| Numpy   	| Tensorflow 	| Cuda    	|
|-------	|---------	|------------	|---------	|
| 256   	| 547 µs  	| 83.7 µs    	| 53 µs   	|
| 1024  	| 32.1 ms 	| 78.1 µs    	| 69.9 µs 	|
| 4096  	| 2.05 s  	| 103 µs     	| 52.9 µs 	|
| 16384 	| 124 s   	| 218 µs     	| 84.4 µs 	|